In [4]:
import pandas as pd
import sqlalchemy
from sqlalchemy.orm import sessionmaker

url = "postgres://events_writer@localhost:5432/events"
readonly_user = "events_reader"
db = sqlalchemy.create_engine(url)

filename = "../datasets/icews/events.1995.csv"
df = pd.read_csv(filename, index_col=0, encoding="utf-8", parse_dates = ['EventDate'])

table_name = "icews"

def primary_key(df):
    return ','.join('"{0}"'.format(i) for i in df.index.names)
    
def table_exists(engine, table_name):
    return engine.dialect.has_table(engine, table_name)
    
if not table_exists(db, table_name):
    Session = sessionmaker(bind=db, autocommit=True)
    session = Session()

    df_empty = df.drop(df.index)
    df_empty.to_sql(table_name, db)
    
    with session.begin():
        session.execute('ALTER TABLE %s ADD PRIMARY KEY (%s);' % (table_name, primary_key(df)))
        session.execute('GRANT SELECT ON %s TO %s;' % (table_name, readonly_user))
    
df_existing = pd.read_sql('SELECT %s from %s' % (primary_key(df), table_name), db)
df_existing = df_existing.set_index(df.index.names)
df_merged = pd.merge(df, df_existing, how="inner", left_index=True, right_index=True, copy=False)

df_new = df.drop(df_merged.index)  

start = 0
end = 5

df_new[start:end].to_sql(table_name, db, if_exists = 'append')

In [26]:
import math
import datetime

import pandas as pd
import sqlalchemy
from sqlalchemy.orm import sessionmaker

filename = "../datasets/icews/events.1995.csv"
df = pd.read_csv(filename, index_col=0, encoding="utf-8", parse_dates = ['EventDate'])

weeksSinceEpoch = [int(math.floor((d - datetime.datetime.utcfromtimestamp(0)).days/7)) for d in df.EventDate]

df['Year'] = df.EventDate.dt.year
df['Month'] = df.EventDate.dt.month
df['Day'] = df.EventDate.dt.day
df['WeeksSinceEpoch'] = weeksSinceEpoch

df.tail()

,EventDate,SourceName,SourceSectors,SourceCountry,EventText,CAMEOCode,Intensity,TargetName,TargetSectors,TargetCountry,...,City,District,Province,Country,Latitude,Longitude,Year,Month,Day,WeeksSinceEpoch
EventID,,,,,,,,,,,,,,,,,,,,,
1094351,1995-12-31,Iran,NaN,Iran,Praise or endorse,51,3.4,Party Member (Iran),Parties,Iran,...,NaN,NaN,NaN,Iran,35.6944,51.4215,1995,12,31,1356
1094352,1995-12-31,Arsenio Farell,"Parties,(National) Major Party,Management / Bu...",Mexico,Accuse,112,-2.0,Citizen (Mexico),"General Population / Civilian / Social,Social",Mexico,...,NaN,NaN,NaN,Mexico,19.4285,-99.1277,1995,12,31,1356
1094353,1995-12-31,Ultima Hora,"Social,Media,Print News,News",Paraguay,Make statement,10,0.0,Túpac Amaru Revolutionary Movement,"Communist,Dissident,Insurgents,Ideological",Peru,...,NaN,NaN,NaN,Peru,-12.0432,-77.0282,1995,12,31,1356
1094354,1995-12-31,Luis Donaldo Colosio,"Center Left,Parties,Ideological,(National) Maj...",Mexico,Make statement,10,0.0,Lawyer/Attorney (Mexico),"Social,Legal",Mexico,...,NaN,NaN,NaN,Mexico,19.4285,-99.1277,1995,12,31,1356
1094355,1995-12-31,Police (Argentina),"Police,Government",Argentina,"Arrest, detain, or charge with legal action",173,-5.0,Citizen (Argentina),"Social,General Population / Civilian / Social",Argentina,...,Buenos Aires,NaN,Ciudad Autonoma de Buenos Aires,Argentina,-34.6132,-58.3772,1995,12,31,1356
